In [1]:
import json

In [2]:
from urllib.parse import unquote

In [35]:
from bs4 import BeautifulSoup
from curl_cffi import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

## Get State Data

In [36]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [37]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Population data

In [38]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [39]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [40]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get REI Data

In [ ]:
url = "https://www.rei.com/stores/map"
headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-language": "en-US,en;q=0.9,ru;q=0.8,el;q=0.7",
    "cache-control": "max-age=0",
    "priority": "u=0, i",
    "sec-ch-ua": '"Chromium";v="140", "Not=A?Brand";v="24", "Google Chrome";v="140"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "cross-site",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "Referer": "https://www.google.com/",
}

raw_cookie = """"""  # Add your cookie from the source in the README.md you need to open dev tools on a browser and copy your cookie here
cookies = dict(item.strip().split("=", 1) for item in raw_cookie.split("; "))

resp = requests.get(
    "https://www.rei.com/stores/map",
    headers=headers,
    cookies=cookies,
    impersonate="chrome110",
)

In [61]:
soup = BeautifulSoup(resp.text, "html.parser")
script = soup.find("script", {"type": "application/json", "id": "modelData"})

In [62]:
resp_dict = json.loads(script.text)
stores = resp_dict["pageData"]["allStores"]["storeLocatorDataQueryModelList"]

In [64]:
store_dicts = []
for store in stores:
    store_dict = store["storeDataModel"]
    if store_dict["countryCode"] == "US":
        store_dicts.append(
            {
                "ID": store_dict["storeNumber"],
                "geometry": Point(store_dict["longitude"], store_dict["latitude"]),
                "STUSPS": store_dict["stateCode"],
                "URL": store_dict["directionLink"],
            }
        )

In [68]:
store_gdf = gpd.GeoDataFrame(store_dicts, crs=4326)

In [69]:
store_gdf = store_gdf.to_crs(9311)
store_gdf.to_file("data/stores.gpkg")

In [70]:
stores_count_df = store_gdf.groupby("STUSPS").size().reset_index(name="COUNT")

## Combine With States

In [75]:
stores_count_gdf = states_with_population_df.merge(
    stores_count_df, on="STUSPS", how="left"
)

In [76]:
stores_count_gdf = stores_count_gdf.fillna(0)

In [77]:
stores_count_gdf["per_100k"] = (
    stores_count_gdf["COUNT"] / (stores_count_gdf["POPULATION"] / 100_000)
).round(decimals=1)
stores_count_gdf["per_1m"] = (
    stores_count_gdf["COUNT"] / (stores_count_gdf["POPULATION"] / 1_000_000)
).round(decimals=1)

In [78]:
stores_count_gdf = stores_count_gdf.fillna(0)

In [79]:
stores_count_gdf = stores_count_gdf.to_crs(9311)
stores_count_gdf.to_file(f"data/REI_Locations_Per_Statae.gpkg")

In [81]:
stores_count_gdf.sort_values("per_1m", ascending=False)

,STUSPS,NAME,POPULATION,geometry,COUNT,per_100k,per_1m
31,MT,Montana,1131302.0,"POLYGON ((-1172889.072 508069.731, -1172583.63...",4.0,0.4,3.5
27,AK,Alaska,736510.0,"MULTIPOLYGON (((-4288407.995 3426181.085, -428...",2.0,0.3,2.7
50,OR,Oregon,4253653.0,"MULTIPOLYGON (((-1781988.816 403834.726, -1781...",9.0,0.2,2.1
21,CO,Colorado,5901339.0,"POLYGON ((-786602.67 -668516.216, -785169.939 ...",11.0,0.2,1.9
24,WY,Wyoming,585067.0,"POLYGON ((-865730.986 59352.206, -864993.905 5...",1.0,0.2,1.7
36,DC,District of Columbia,687324.0,"POLYGON ((1950799.304 -402452.215, 1951231.574...",1.0,0.1,1.5
30,VT,Vermont,648708.0,"POLYGON ((2075870.027 238820.542, 2075640.482 ...",1.0,0.2,1.5
11,WA,Washington,7857320.0,"MULTIPOLYGON (((-1634509.823 627551.89, -16343...",11.0,0.1,1.4
34,NH,New Hampshire,1402199.0,"MULTIPOLYGON (((2326343.717 202530.281, 232642...",2.0,0.1,1.4
43,DE,Delaware,1036423.0,"MULTIPOLYGON (((2060773.554 -301785.14, 206090...",1.0,0.1,1.0
